<a href="https://colab.research.google.com/github/sana-f-shah/Smart-Meter-Forecasting/blob/main/notebooks/2_creating_data_versions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
original = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/regression/original.parquet'
df = pd.read_parquet(original)

In [ ]:
# Regression
"""
test_df = df.sample(n=100, random_state=42)
train_df = df.drop(test_df.index)

test_path = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/regression/test.parquet'
test_df.to_parquet(test_path, index=False)

train_path = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/regression/original.parquet'
train_df.to_parquet(train_path, index=False)

output_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/regression/'
numeric_cols = df.select_dtypes(include=[np.number]).columns
"""

"\ntest_df = df.sample(n=100, random_state=42)\ntrain_df = df.drop(test_df.index)\n\ntest_path = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/data_versions/regression/test.parquet'\ntest_df.to_parquet(test_path, index=False)\n\ntrain_path = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/data_versions/regression/original.parquet'\ntrain_df.to_parquet(train_path, index=False)\n\noutput_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/data_versions/regression/'\nnumeric_cols = df.select_dtypes(include=[np.number]).columns\n"

In [ ]:
# Time Series

df['data_collection_log_timestamp'] = pd.to_datetime(df['data_collection_log_timestamp'])
df = df.sort_values('data_collection_log_timestamp')
df.set_index('data_collection_log_timestamp', inplace=True)
df['target'] = df['total_consumption_active_import']
df['lag_1'] = df['target'].shift(1)
df['lag_2'] = df['target'].shift(2)
df['rolling_3h'] = df['target'].rolling(window=3).mean()
df = df.dropna()

test_df = df.sample(n=100, random_state=42)
train_df = df.drop(test_df.index)

test_path = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/time series/test.parquet'
test_df.to_parquet(test_path, index=False)

train_path = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/time series/original.parquet'
train_df.to_parquet(train_path, index=False)

output_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Forecasting/data_versions/time series/'
numeric_cols = df.select_dtypes(include=[np.number]).columns

In [ ]:
df_outlier_removed = df.copy()
for col in numeric_cols:
    q1 = df_outlier_removed[col].quantile(0.25)
    q3 = df_outlier_removed[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df_outlier_removed = df_outlier_removed[(df_outlier_removed[col] >= lower) & (df_outlier_removed[col] <= upper)]
df_outlier_removed.to_parquet(f'{output_dir}/no_outliers.parquet', index=False)

In [ ]:
from scipy.stats.mstats import winsorize

df_winsorized = df.copy()
for col in numeric_cols:
    df_winsorized[col] = winsorize(df[col], limits=[0.01, 0.01])
df_winsorized.to_parquet(f'{output_dir}/winsorized.parquet', index=False)

In [ ]:
"""
df_outlier_flags = df.copy()
for col in numeric_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df_outlier_flags[f'is_outlier_{col}'] = ((df[col] < lower) | (df[col] > upper)).astype(int)
df_outlier_flags.to_parquet(f'{output_dir}/with_outlier_flags.parquet', index=False)
"""

In [ ]:
from sklearn.impute import SimpleImputer

df_imputed = df.copy()
imputer = SimpleImputer(strategy='median')
df_imputed[numeric_cols] = imputer.fit_transform(df[numeric_cols])
df_imputed.to_parquet(f'{output_dir}/imputed.parquet', index=False)